<a href="https://colab.research.google.com/github/Shrondi/smart-bridges/blob/main/visualizer_smartbridges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importación e instalación

In [159]:
!pip install -q pandas
!pip install -q matplotlib
!pip install -q plotly

In [160]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

# Definición funciones

In [136]:
def calc_offsets(df):

    offsets = {}  # Diccionario

    # Calcular y aplicar offset para cada acelerómetro
    for acc_num in df['accelerometer'].unique():
        df_acc = df[df['accelerometer'] == acc_num]

        # Calcular la media de cada eje para este acelerómetro
        mean_x = df_acc['x'].mean()
        mean_y = df_acc['y'].mean()
        mean_z = df_acc['z'].mean()

        # Ajustar los datos restando la media y sumando 1 en Z
        df.loc[df['accelerometer'] == acc_num, 'x'] = df_acc['x'] - mean_x
        df.loc[df['accelerometer'] == acc_num, 'y'] = df_acc['y'] - mean_y
        df.loc[df['accelerometer'] == acc_num, 'z'] = df_acc['z'] - mean_z + 1

        # Store the calculated offsets in the dictionary
        offsets[acc_num] = {'x': -mean_x, 'y': -mean_y, 'z': -(mean_z - 1)}

    return offsets

In [188]:
def plot_accelerometer_data(df, offsets):

    fig, (axis_x, axis_y, axis_z, axis_table) = plt.subplots(4, 1, figsize=(10, 18))

    # Obtener el primer datetime para usarlo como titulo
    first_datetime = df.index[0]

    formatted_date = first_datetime.strftime("%d/%m")  # Dia/Mes
    formatted_time = first_datetime.strftime("%H:%M:%S")  # Hora:Minutos:Segundos

    # Crear titulo de la figura
    figure_title = f"Tren {formatted_date} {formatted_time}"
    fig.suptitle(figure_title, fontsize=16, fontweight='bold')

    fig.subplots_adjust(hspace=0.5, top=0.92)

    # Paleta de colores para los acelerometros
    palette = sns.color_palette('bright', n_colors=8)

    # Create a color map using the palette
    colors = {
        1: {'x': palette[0], 'y': palette[1], 'z': palette[2]},
        2: {'x': palette[3], 'y': palette[4], 'z': palette[5]},
        3: {'x': palette[6], 'y': palette[7], 'z': palette[0]},
        4: {'x': palette[1], 'y': palette[2], 'z': palette[3]},
        5: {'x': palette[4], 'y': palette[5], 'z': palette[6]},
        6: {'x': palette[7], 'y': palette[0], 'z': palette[1]},
        7: {'x': palette[2], 'y': palette[3], 'z': palette[4]},
        8: {'x': palette[5], 'y': palette[6], 'z': palette[7]}
    }

    # Plotear datos
    for acc_num in df['accelerometer'].unique():
        df_acc = df[df['accelerometer'] == acc_num]
        df_acc = df_acc[['x', 'y', 'z']]

        # Calculate time difference in seconds
        time_diff = (df_acc.index - df_acc.index[0]).to_series().dt.total_seconds()

        # Access color using accelerometer number and axis
        axis_x.plot(time_diff, df_acc['x'], label=f'Acelerómetro {acc_num}', color=colors.get(acc_num, {}).get('x'))
        axis_y.plot(time_diff, df_acc['y'], label=f'Acelerómetro {acc_num}', color=colors.get(acc_num, {}).get('y'))
        axis_z.plot(time_diff, df_acc['z'], label=f'Acelerómetro {acc_num}', color=colors.get(acc_num, {}).get('z'))

    # Configurar ejes para cada gráfica
    for axis in [axis_x, axis_y, axis_z]:
        axis.set_xlabel('Tiempo [s]', fontsize=12)
        axis.set_ylabel('Aceleración [g]', fontsize=12)
        axis.legend(loc="lower left")
        axis.grid(which='major', linestyle='--', alpha=0.5)
        axis.grid(which='minor', linestyle=':', alpha=0.3)
        axis.autoscale()

    axis_x.set_title('Acc. X', fontsize=14, fontweight='bold')
    axis_y.set_title('Acc. Y', fontsize=14, fontweight='bold')
    axis_z.set_title('Acc. Z', fontsize=14, fontweight='bold')

    table_data = [
        [f'{offsets[acc_num]["x"]:.4f}' for acc_num in offsets],  # X Offsets
        [f'{offsets[acc_num]["y"]:.4f}' for acc_num in offsets],  # Y Offsets
        [f'{offsets[acc_num]["z"]:.4f}' for acc_num in offsets]   # Z Offsets
    ]

    # Add table as a subplot
    axis_table.axis('off')
    table = axis_table.table(cellText=table_data,
                    colLabels=[f'Acelerómetro {acc_num}' for acc_num in offsets],  # Column labels are accelerometer numbers
                    rowLabels=['X', 'Y', 'Z'],  # Row labels are X, Y, Z
                    cellLoc='center',
                    loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(0.6, 1.5)  # Adjust scale as needed
    axis_table.set_title("Offsets acelerómetros", y=0.70, fontweight='bold')  # Add title above the table

    return fig, plt

In [167]:
def load_data(input_path):
    """Loads and processes a single CSV file."""
    print(f"Cargando datos: {input_path}")
    try:
        df = pd.read_csv(input_path, index_col='datetime', parse_dates=['datetime'],
                         date_format='%Y-%m-%d %H:%M:%S.%f', dayfirst=True)

        df.sort_index(inplace=True)

    except Exception as e:
        print(f"Error al procesar el archivo {input_path}: {e}")
        return pd.DataFrame()

    return df

In [169]:
def process_train_file(filepath, show=True, save=False, output_dir='./', file='trains.pdf'):
    """
    Plots accelerometer data from a CSV file or a directory of CSV files.

    Args:
        filepath (str): The path to the CSV file or directory.
    """
    if save:
      pdf = PdfPages(os.path.join(output_dir, file))

    for filename in sorted([f for f in os.listdir(filepath) if f.endswith(".csv")]):

        plt = None
        fig = None

        df = load_data(os.path.join(filepath, filename))

        if not df.empty:
            offsets = calc_offsets(df)
            fig, plt = plot_accelerometer_data(df, offsets)

            if show:
                plt.show()

            if save and fig is not None:

                if not os.path.exists(output_dir):
                  os.makedirs(output_dir)

                pdf.savefig(fig)

                print(f"Figura guardada en {os.path.join(output_dir, file)}") # Print the file path

            plt.close()

        else:
            print(f"No se encontraron datos en: {os.path.join(filepath, filename)}")

    if save:
      pdf.close()

# Resultados

In [ ]:
process_train_file('Escritorio/SMART-BRIDGES/Guadiato/', save=True, show=True)